# Converting raw .txt files from labview to fretbursts compatible HDF5 files

This notebook will convert the .txt files written by our labview acquisition software into HDF5 files which can be read into other FRETbursts notebooks. 

It also reads in the metadata file generated by labview and uses the information in it to build the HDF5 file.

In [1]:
import phconvert as phc
import csv
phc.__version__
import numpy as np
import os

# Read in data

Name the directory of the raw txt file, the metadatafile, and what you want to save the hdf5 file as

In [3]:
raw = 'definitiveset/0001/001_1A1_TXT.txt'
metadata = 'definitiveset/0001/metadata.txt'
savename = 'definitiveset/1a1test.hdf5'

In [4]:
try:
    with open(raw): pass
    print('Data file found, you can proceed.')
except IOError:
    print('ATTENTION: Data file not found, please check the filename.\n'
          '           (current value "%s")' % raw)

Data file found, you can proceed.


The following code will read in the txt and split it into separate arrays for the timestamps and detector of each photon. This can take a couple of seconds to execute depending on the size of the file.

In [5]:
with open(raw) as inf:
    reader = csv.reader(inf, delimiter="	")
    timestamps = list(zip(*reader))[0]
with open(raw) as inf:
    reader = csv.reader(inf, delimiter="	")
    detectors = list(zip(*reader))[1]
detectors = np.asarray(detectors)
detectors = np.uint8(detectors)
timestamps = np.asarray(timestamps)
timestamps = np.int64(timestamps)

In [16]:
with open(metadata) as inf:
    reader = csv.reader(inf, delimiter=":")
    metastuff = list(zip(*reader))
metastuff[0]
User = metastuff[1][3]
Acquitime = metastuff[1][0]
Dpower = metastuff[1][1]
Apower = metastuff[1][2]
Fluorconc = metastuff[1][4]
Sampdetails = metastuff[1][5]
Donor = metastuff[1][6]
Acceptor = metastuff[1][7]
buffer = metastuff[1][8]
Notes = metastuff[1][9]

# Create metadata

In [29]:
description = Notes

author = User
author_affiliation = "University of Sheffield" #Change this as necessary...

sample_name = Sampdetails
buffer_name = buffer
dye_names = DYES  # Comma separates names of fluorophores

# Create Photon-HDF5 data structure

In [30]:
timestamps_unit = 10e-9  # 10 ns, units are always S.I.
photon_data = dict(
    timestamps=timestamps,
    detectors=detectors,
    timestamps_specs={'timestamps_unit': timestamps_unit})
setup = dict(
    ## Mandatory fields
    num_pixels = 2,                   # using 2 detectors
    num_spots = 1,                    # a single confoca excitation
    num_spectral_ch = 2,              # donor and acceptor detection 
    num_polarization_ch = 1,          # no polarization selection 
    num_split_ch = 1,                 # no beam splitter
    modulated_excitation = False,     # CW excitation, no modulation 
    excitation_alternated = [True],  # CW excitation, no modulation 
    lifetime = False,                 # no TCSPC in detection    
    ## Optional fields
    excitation_wavelengths = [515e-9, 638e-9],         # List of excitation wavelenghts
    excitation_cw = [True],                    # List of booleans, True if wavelength is CW
    detection_wavelengths = [580e-9, 690e-9],  # Nominal center wavelength 
                                               # each for detection ch
)
provenance = dict(
    filename=raw, 
    software='Custom Labview Software - Craggs Lab')
identity = dict(
    author=author,
    author_affiliation=author_affiliation)
measurement_specs = dict(
    measurement_type = 'smFRET-usALEX',
    detectors_specs = {'spectral_ch1': [0],  # list of donor's detector IDs
                       'spectral_ch2': [1]},  # list of acceptor's detector IDs,
    alex_period = 10000,
    )
photon_data['measurement_specs'] = measurement_specs

data = dict(
    description=description,
    photon_data = photon_data,
    setup=setup,
    identity=identity,
    provenance=provenance
)
phc.hdf5.save_photon_hdf5(data, h5_fname=savename, overwrite=True)
print("Done")